In [1]:
#export
from nbexp_personal import sendEmail

## utils

In [2]:
#export
def itemgetter(*args):
    g = operator.itemgetter(*args)
    def f(*args2):
        return dict(zip(args,  g(*args2)))
    return f

In [3]:
#export
def write_json(filename, content):
    with open(filename, 'w', encoding='UTF-8') as f:
        json.dump( content,f, ensure_ascii=False, indent=4)
def read_json(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        return json.load( f)

## read and convert 

In [3]:
#export
# 复制为cCURL（posix）
with open('../bili_curl.txt', 'r', encoding='UTF-8') as f:
    code = f.read()

In [4]:
code

'curl "https://api.vc.bilibili.com/dynamic_svr/v1/dynamic_svr/dynamic_new?uid=362761&type_list=8,512,4097,4098,4099,4100,4101" -H "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0" -H "Accept: application/json, text/plain, */*" -H "Accept-Language: zh-CN,ja;q=0.8,zh;q=0.6,en-US;q=0.4,en;q=0.2" --compressed -H "Origin: https://t.bilibili.com" -H "DNT: 1" -H "Connection: keep-alive" -H "Referer: https://t.bilibili.com/pages/nav/index_new" -H "Cookie: l=v; _uuid=EEE030C1-6229-6843-C00A-FE757FF1F43368764infoc; buvid3=C96EF2FC-7530-485C-ACB1-BDE4CBC21A42190961infoc; LIVE_BUVID=AUTO5515748557684260; CURRENT_FNVAL=16; stardustvideo=1; rpdid=|()kll|JJum0J\'ul~lJllJkm; bp_t_offset_362761=362370857533697408; laboratory=1-1; im_notify_type_362761=0; CURRENT_QUALITY=32; DedeUserID=362761; DedeUserID__ckMd5=494ab7df9fc9a62f; SESSDATA=8e587874"%"2C1584615636"%"2C4a935d21; bili_jct=356ced437bcbce1de069ab6c523170e4; INTVER=1; sid=9y6gyzn1; stardustpgcv=0606; m

In [6]:
# !{code.strip()}

In [6]:
#export
import uncurl
import requests
from functools import partial
def fetch_code(code):
    c = uncurl.parse(code)
    r = eval(c)
    j = r.json()
    return j

In [42]:
#export
import operator
import json
import datetime
def get_time(timestamp):
    d = datetime.datetime.fromtimestamp(timestamp)
    d = d.isoformat()
    return d
    
def cvt_cards(j):
    cards = j['data']['cards']
    cards = list(map(operator.itemgetter('card'), cards))
    cards = list(map(json.loads, cards))
    kl = ('title', 'desc',  'pic', 'stat', 'ctime')
    cards = list(map(itemgetter(*kl), cards))
    def cvt(card):
        content_id = str(card['stat']['aid'])
        content = itemgetter(*kl[:-2])(card)
        pic = content['pic'] + '@64w_36h_1c.jpg'
        content['pic'] = pic
        d = get_time(card['ctime'])
        url = 'https://www.bilibili.com/video/av' + content_id 
        return (content_id,  {'content': content , "url":url , 'time':d} )
    cards = dict((map(cvt, cards)))
    return cards

## render

In [8]:
#export
def render_div(v):
    content = v['content']
    desc = content['desc']

    if len(desc) > 50:
        desc = desc[:20]+'...'
    
    body = f"""
        <div style="margin:10px">
            <img src='{content['pic']}'>
            <a href='{v['url']}'>{content['title']}</a>
            <span>{desc} {v['time']}</span>
        </div>
    """
    return body

In [9]:
#export
def render_html(v_list):
    divs = ''.join(map(render_div, v_list))
    html = f"""\
    <html>
      <head></head>
      <body>
          {divs}
      </body>
    </html>
    """
    return html

In [54]:
#export
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Create the body of the message (a plain-text and an HTML version).
def render_msg(v_list, sub_name=""):
    v_list = list(v_list)
    html = render_html(v_list)
    msg = MIMEMultipart('alternative')
    msg['Subject'] = sub_name + '订阅' + '+' + str(len(v_list))
    msg['From'] = sub_name
    msg['To'] = ''
    # Record the MIME types of both parts - text/plain and text/html.
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    return msg.as_string()

In [11]:
# msg = render_msg(cards.values())

## check oldcards

In [52]:
#export
def get_cards():
    fetch = partial(fetch_code, code)
    cards = cvt_cards(fetch())
    return cards
    
def get_main(json_path, get_cards, sub_name=""):
    
    def main():
        cards = get_cards()
        wj = partial( write_json, json_path,)
        rj = partial( read_json, json_path,)

        if not exists(json_path):
            wj(cards)

        old_cards = rj()

        new_cards = filter(lambda tp:tp[0] not in old_cards, cards.items())
        new_cards = map(operator.itemgetter(1), new_cards)
        new_cards = list(new_cards)
        if new_cards:
            msg = render_msg(new_cards, sub_name)
            sendEmail(msg)
            old_cards.update(cards)
            wj(old_cards)
            
    return main

In [53]:
#export
from os.path import exists
json_path = './bili.json'
main = get_main(json_path, get_cards, "bili")


In [35]:
main()

In [8]:
!python notebook2script.py bilibili.ipynb

Converted bilibili.ipynb to nbexp_bilibili.py


In [56]:
rj = partial( read_json, json_path,)
cards = rj()
new_cards = map(operator.itemgetter(1), cards.items())
new_cards = list(new_cards )
msg = render_msg(new_cards, 'bili')
sendEmail(msg, )

In [ ]:
new_cards[0]

In [16]:
r = fetch_code(code)